In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
#import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math
import dill as pickle

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.utils import get_aligned_ids
from mcmodels.core import VoxelModelCache
from mcmodels.connectivity.creation import get_connectivity_matrices3
from mcmodels.core.connectivity_data import get_connectivity_data, ConnectivityData
from mcmodels.core.structure_data import StructureData
from mcmodels.core.model_data import ModelData
#from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures, get_leaves_ontologicalorder, get_indices_2ormore, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3
from mcmodels.regressors import NadarayaWatson
from mcmodels.core.plotting import plot_loss_surface,plot_loss_scatter
from mcmodels.models.expectedloss.crossvalidation import get_loss_surface_cv_spline
from mcmodels.models.expectedloss.crossvalidation import get_embedding_cv
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss, Crossval
from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists

def get_row_col_names(connectivity_data, target_ordering):
    
    rnames = np.asarray([ia_map[target_ordering[i]] for i in range(len(target_ordering))])
    ipsi_target_regions = connectivity_data.ipsi_target_regions
    contra_target_regions = connectivity_data.contra_target_regions                               
    ipsi_indices= np.asarray([])
    contra_indices = np.asarray([])
    for iy in target_ordering: 
        ipsi_indices = np.concatenate([ipsi_indices, np.where(ipsi_target_regions==iy)[0]] )
        contra_indices = np.concatenate([contra_indices, np.where(contra_target_regions==iy)[0]] )
    ipsi_indices = np.asarray(ipsi_indices, dtype = int)   
    contra_indices = np.asarray(contra_indices, dtype = int)    
    reorder = np.concatenate([ipsi_indices, len(ipsi_indices) + contra_indices])  
    colids = np.concatenate([ipsi_target_regions, contra_target_regions])[reorder]
    cnames = np.asarray([ia_map[colids[i]] for i in range(len(colids))])
    ccomb = np.vstack([np.concatenate([np.repeat('ipsi',connectivity_data.ipsi_target_regions.shape[0]),
                                       np.repeat('contra',connectivity_data.contra_target_regions.shape)]), cnames])
    ccomb = np.asarray(ccomb)
    tuples2 = list(zip(*ccomb))
    cnam_multi = pd.MultiIndex.from_tuples(tuples2, names=['first', 'second'])
    return(cnam_multi, rnames)

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
input_data = ju.read(INPUT_JSON)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
cache = VoxelModelCache(manifest_file=manifest_file)
st = cache.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}
major_structures = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/major_structures.npy')
major_structure_ids = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/major_structure_ids.npy')
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)

annotation/ccf_2017


In [16]:
connectivity_data = ConnectivityData(cache)

In [19]:
sid = 512
sid_data = StructureData(sid)
structure_set_id=167587189

In [20]:
model_data = ModelData(cache, sid, structure_set_id)

In [22]:
np.asarray(list(model_data.get_experiment_ids(experiments_exclude=experiments_exclude, cre=None))).shape

(36,)

In [4]:
experiment_ids = [e['id'] for e in cache.get_experiments(
            dataframe=False, cre=None, injection_structure_ids=[512])]

In [10]:
cache.get_experiments(
            dataframe=False, cre=None, injection_structure_ids=[512])

[{'transgenic_line': 'Pcp2-Cre_GN135',
  'gender': 'M',
  'injection_structures': [1017],
  'injection_volume': 0.1692286344,
  'injection_x': 12170,
  'injection_y': 3140,
  'injection_z': 8540,
  'product_id': 5,
  'specimen_name': 'Pcp2-Cre (NH)-3624',
  'strain': 'C57BL/6J',
  'structure_abbrev': 'AN',
  'structure_id': 1017,
  'structure_name': 'Ansiform lobule',
  'primary_injection_structure': 1017,
  'id': 168230532},
 {'transgenic_line': 'Htr2a-Cre_KM207',
  'gender': 'M',
  'injection_structures': [96, 846, 1007, 1017, 1033, 1041],
  'injection_volume': 0.00645919225,
  'injection_x': 11160,
  'injection_y': 3220,
  'injection_z': 8380,
  'product_id': 5,
  'specimen_name': 'Htr2a-Cre_KM207-130963',
  'strain': 'unknown',
  'structure_abbrev': 'DN',
  'structure_id': 846,
  'structure_name': 'Dentate nucleus',
  'primary_injection_structure': 96,
  'id': 552283801},
 {'transgenic_line': 'Gabra6-IRES-Cre',
  'gender': 'M',
  'injection_structures': [1007, 1017],
  'injection_v

In [16]:
cache.manifest_file

'/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/new_manifest.json'

In [6]:
len(experiment_ids)

36

In [8]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [9]:
help(MouseConnectivityCache)

Help on class MouseConnectivityCache in module allensdk.core.mouse_connectivity_cache:

class MouseConnectivityCache(allensdk.core.reference_space_cache.ReferenceSpaceCache)
 |  MouseConnectivityCache(resolution=None, cache=True, manifest_file=None, ccf_version=None, base_uri=None, version=None)
 |  
 |  Cache class for storing and accessing data related to the adult mouse
 |  Connectivity Atlas.  By default, this class will cache any downloaded
 |  metadata or files in well known locations defined in a manifest file.
 |  This behavior can be disabled.
 |  
 |  Attributes
 |  ----------
 |  
 |  resolution: int
 |      Resolution of grid data to be downloaded when accessing projection volume,
 |      the annotation volume, and the annotation volume.  Must be one of (10, 25,
 |      50, 100).  Default is 25.
 |  
 |  api: MouseConnectivityApi instance
 |      Used internally to make API queries.
 |  
 |  Parameters
 |  ----------
 |  
 |  resolution: int
 |      Resolution of grid data 

In [ ]:
get_experiments(self, dataframe=False, file_name=None, cre=None, injection_structure_ids=None)

In [38]:
from allensdk.api.cache import Cache

In [39]:
help(Cache)

Help on class Cache in module allensdk.api.cache:

class Cache(builtins.object)
 |  Cache(manifest=None, cache=True, version=None, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, manifest=None, cache=True, version=None, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add_manifest_paths(self, manifest_builder)
 |      Add cache-class specific paths to the manifest. In derived classes,
 |      should call super.
 |  
 |  build_manifest(self, file_name)
 |      Creation of default path specifications.
 |      
 |      Parameters
 |      ----------
 |      file_name : string
 |          where to save it
 |  
 |  get_cache_path(self, file_name, manifest_key, *args)
 |      Helper method for accessing path specs from manifest keys.
 |      
 |      Parameters
 |      ----------
 |      file_name : string
 |      manifest_key : string
 |      args : ordered parameters
 |      
 |      Returns
 |      -------
 |      string or None
 | 

In [25]:
import allensdk

In [27]:
from allensdk.api.cache import cacheable, Cache

In [32]:
help(allensdk.api.cache)

Help on module allensdk.api.cache in allensdk.api:

NAME
    allensdk.api.cache

DESCRIPTION
    # Allen Institute Software License - This software license is the 2-clause BSD
    # license plus a third clause that prohibits redistribution for commercial
    # purposes without further permission.
    #
    # Copyright 2015-2017. Allen Institute. All rights reserved.
    #
    # Redistribution and use in source and binary forms, with or without
    # modification, are permitted provided that the following conditions are met:
    #
    # 1. Redistributions of source code must retain the above copyright notice,
    # this list of conditions and the following disclaimer.
    #
    # 2. Redistributions in binary form must reproduce the above copyright notice,
    # this list of conditions and the following disclaimer in the documentation
    # and/or other materials provided with the distribution.
    #
    # 3. Redistributions for commercial purposes are not permitted without the
    # All

In [ ]:

def get_connectivity_data(cache, structure_ids, experiments_exclude, remove_injection=False, structure_set_id = 687527945):
    '''

    :param cache: VoxelModelCache for communicating with AllenSDK
    :param structure_ids: Ids for which to pull data
    :param experiments_exclude: Experiments to exclude (not tested)
    :param remove_injection: Remove injection signal from projection (not tested)
    :param structure_set_id: Structure set specifying list of structures (not same as location of structures)
    :return:
    '''
    connectivity_data = ConnectivityData(cache)

    for sid in structure_ids:
        print(sid)
        sid_data = StructureData(sid)
        # deprecated language
        model_data = ModelData(cache, sid, structure_set_id)
        sid_data.eids = model_data.get_experiment_ids(experiments_exclude=experiments_exclude, cre=None)
 